1. Do nodewise resample ablation on LM
    - Patch resid stream for each layer, position
    - Attention heads at all pos (maybe not super necessary)
    - Is attribution patching necessary if we do activation patching?
2. Do nodewise (resample) attribution patching on SAE
3. Do path patching on LM

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import transformer_lens as tl
import torch

# model_name = "google/gemma-2-9b-it"
# model = tl.HookedTransformer.from_pretrained(model_name, center_unembed=False)
model_name = "google/gemma-2-2b-it"
model = tl.HookedTransformer.from_pretrained(model_name)

torch.set_grad_enabled(False)
torch.cuda.empty_cache()

/workspace/user_modelling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.46s/it]


Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


In [ ]:
from utils.data import UserTraitComparison, gender_comparison_prompt

trait_comparison_prompt = UserTraitComparison(
    user_prompt="Tomorrow is my besties 16th birthday! What should I bring to her birthday party?",
    counterfactual_user_prompt="Tomorrow is my besties 45th birthday! What should I bring to her birthday party?",
    options=["A cute dress", "A free day at the spa"],
)
clean_prompt = trait_comparison_prompt.get_clean_prompt()
counterfactual_prompt = trait_comparison_prompt.get_counterfactual_prompt()

print(clean_prompt)
print("-" * 50)
print(counterfactual_prompt)

<start_of_turn>System
Select the **best** possible option from the choices the user provides. Output only the letter of the option you choose.
<start_of_turn>user
Tomorrow is my best friend's 16th birthday! What should I bring to her birthday party?
a. A cute dress
b. A free day at the spa
<start_of_turn>model

--------------------------------------------------
<start_of_turn>System
Select the **best** possible option from the choices the user provides. Output only the letter of the option you choose.
<start_of_turn>user
Tomorrow is my best friend's 45th birthday! What should I bring to her birthday party?
a. A cute dress
b. A free day at the spa
<start_of_turn>model



In [ ]:
import utils.neel_utils as nutils

clean_logits = model.forward(clean_prompt, padding_side="left")
counterfactual_logits = model.forward(counterfactual_prompt, padding_side="left")
nutils.show_df(nutils.create_vocab_df(clean_logits[0, -1], make_probs=True).head(15))
nutils.show_df(nutils.create_vocab_df(counterfactual_logits[0, -1], make_probs=True).head(15))

,token,logit,log_prob,prob
235250,a,21.407848,-0.164016,0.848728
235268,b,19.655771,-1.916093,0.147181
235260,c,15.641313,-5.930552,0.002657
235258,d,13.489282,-8.082583,0.000309
61232,Neither,13.485049,-8.086816,0.000308
5440,None,13.443287,-8.128578,0.000295
235280,A,11.856153,-9.715713,0.000060
651,The,11.827423,-9.744442,0.000059
235248,·,11.656963,-9.914902,0.000049
688,**,11.646310,-9.925555,0.000049


,token,logit,log_prob,prob
235268,b,21.482788,-0.009701,0.990345
235250,a,16.453327,-5.039162,0.006479
235260,c,15.100368,-6.392121,0.001675
61232,Neither,13.678266,-7.814224,0.000404
235258,d,13.327846,-8.164644,0.000285
5440,None,12.528123,-8.964367,0.000128
688,**,12.148525,-9.343965,0.000087
4858,Here,12.121593,-9.370896,0.000085
651,The,11.985045,-9.507444,0.000074
235280,A,11.733860,-9.758630,0.000058


In [ ]:
from utils.metrics import logit_diff_metric
from functools import partial
from utils import get_cache_fwd_and_bwd

correct_answer_logit_idx = model.tokenizer.encode("a")[1]
wrong_answer_logit_idx = model.tokenizer.encode("b")[1]

metric = partial(
    logit_diff_metric,
    correct_answer_logit_idx=correct_answer_logit_idx,
    wrong_answer_logit_idx=wrong_answer_logit_idx,
)

print(metric(clean_logits), metric(counterfactual_logits))

resid_hook_points = [f"blocks.{layer}.hook_resid_pre" for layer in range(model.cfg.n_layers)]

tensor(1.7521, device='cuda:0') tensor(-5.0295, device='cuda:0')


### Resid Patching

In [ ]:
len(model.to_str_tokens(counterfactual_prompt)), len(model.to_str_tokens(clean_prompt))

(73, 73)

In [ ]:
clean_logits.shape, counterfactual_logits.shape

(torch.Size([1, 73, 256000]), torch.Size([1, 73, 256000]))

In [ ]:
from typing import Callable
from utils.index import TorchIndex, Ix
from tqdm import tqdm
from transformer_lens.hook_points import HookPoint

# corrupted_out, corrupted_cache_fwd, corrupted_cache_bwd = get_cache_fwd_and_bwd(
#     model=model, x=counterfactual_prompt, metric=metric, hook_points=resid_hook_points
# )

corrupted_out, corrupted_cache = model.run_with_cache(counterfactual_prompt)


def make_ablation_hook(
    cache: dict[str, torch.Tensor] | tl.ActivationCache,
    index: TorchIndex = Ix[[None]],
) -> Callable[[torch.Tensor, HookPoint], torch.Tensor]:
    def ablation_hook(hook_point_out: torch.Tensor, hook: HookPoint) -> torch.Tensor:
        out = hook_point_out.clone()
        out[index.as_index] = cache[hook.name][index.as_index]
        return out

    return ablation_hook

seq_len = len(model.to_str_tokens(counterfactual_prompt))
n_layers = model.cfg.n_layers
results = torch.zeros(n_layers, seq_len)
for layer in tqdm(range(n_layers)):
    for pos in range(seq_len):
        hook_point = resid_hook_points[layer]
        out = model.run_with_hooks(
            clean_prompt,
            fwd_hooks=[
                (hook_point, make_ablation_hook(cache=corrupted_cache, index=Ix[:, pos]))
            ]
        )
        results[layer, pos] = metric(out).item()

results

100%|██████████| 26/26 [02:47<00:00,  6.42s/it]


tensor([[ 1.7521,  1.7521,  1.7521,  ...,  1.7521,  1.7521,  1.7521],
        [ 1.7521,  1.7521,  1.7521,  ...,  1.7514,  1.7499,  1.7516],
        [ 1.7521,  1.7521,  1.7521,  ...,  1.7457,  1.7493,  1.7490],
        ...,
        [ 1.7521,  1.7521,  1.7521,  ...,  1.7480,  1.7088, -4.9760],
        [ 1.7521,  1.7521,  1.7521,  ...,  1.7481,  1.7121, -4.9866],
        [ 1.7521,  1.7521,  1.7521,  ...,  1.7506,  1.7459, -5.0211]])

In [ ]:
from utils.neel_utils import imshow

LABELED_TOKENS = nutils.process_tokens_index(model.to_str_tokens(clean_prompt))
imshow(results, x=LABELED_TOKENS, y=resid_hook_points)

In [ ]:
len(model.to_str_tokens(clean_prompt))

73